# Run Super-resolution Model 
This notebook should be moved into the ``Waifu2x`` directory when ready to use.

In [ ]:
import os
from pathlib import Path

import torch
from torch import nn
from torchvision.utils import save_image
from tqdm.auto import tqdm

from Models import CARN_V2
from utils.prepare_images import Image, ImageSplitter, to_tensor

In [ ]:
SOURCE_DIR = (
    'PATH/TO/LIGHTWEIGHT/GAN/GENERATED/DIRECTORY'
)

In [ ]:
save_path = (
    (Path(SOURCE_DIR).parent / 'generated_images_super_resolution')
)
save_path.mkdir(parents=True, exist_ok=True)
save_path = str(save_path)


save_path

In [ ]:
model = CARN_V2(color_channels=3,
                mid_channels=64,
                conv=nn.Conv2d,
                single_conv_size=3,
                single_conv_group=1,
                scale=2,
                activation=nn.LeakyReLU(0.1),
                SEBlock=True,
                repeat_blocks=3,
                atrous=(1, 1, 1))

In [ ]:
checkpoint = 'ape_model/CARN_model_checkpoint.pt'
model.load_state_dict(torch.load(checkpoint, 'cuda'))
model = model.cuda()
model = model.float()
model = model.eval()

In [ ]:
for image_path in tqdm(os.listdir(SOURCE_DIR)):
    img = Image.open(os.path.join(SOURCE_DIR, image_path)).convert('RGB')

    img_t = to_tensor(img).unsqueeze(0).cuda()

    img_upscale = model(img_t)

    save_image(img_upscale, os.path.join(save_path, image_path))

----- 